In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/egiptura-database/Egiptura Database.pdf


In [2]:
# حل التعارض: datasets==3.6.0 بتحتاج fsspec[http]==2023.6.0
!pip -q install -U "fsspec[http]==2023.6.0" "datasets==3.6.0"

# ثبّت الباقي بدون كسر الاعتمادات
!pip -q install -U "transformers>=4.43,<5" "accelerate>=0.33,<1" \
                   "sentence-transformers>=2.5,<3" "faiss-cpu>=1.7,<2" \
                   "pypdf>=3,<5" "safetensors>=0.4.2" "huggingface_hub>=0.23,<1"

# تأكيد الإصدارات (اختياري)
import importlib, pkgutil
for m in ["fsspec","datasets","transformers","accelerate","sentence_transformers","faiss","pypdf"]:
    try:
        mod = importlib.import_module(m if m!="faiss" else "faiss")
        print(m, getattr(mod, "__version__", ""))
    except Exception as e:
        print(m, "ERR:", e)


fsspec 2023.6.0
datasets 3.6.0
transformers 4.56.2
accelerate 0.34.2
sentence_transformers 2.7.0
faiss 1.12.0
pypdf 4.3.1


In [3]:
# If needed (Kaggle/Colab): uncomment & run
%pip install -q  torch accelerate bitsandbytes fastapi uvicorn nest_asyncio pyngrok requests

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install -U bitsandbytes

Note: you may need to restart the kernel to use updated packages.


In [5]:
# لو الريبو Private
from huggingface_hub import login
login()  # هيفتح لك إدخال التوكن



In [6]:
import torch, os, re
from transformers import AutoTokenizer, AutoModelForCausalLM

REPO_ID = "HatemAhmed44/egiptura-adapter"   # عدّلها لو اختلفت

tok = AutoTokenizer.from_pretrained(REPO_ID, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    REPO_ID,
    trust_remote_code=True,
    torch_dtype=(torch.float16 if torch.cuda.is_available() else torch.float32),
    device_map=("auto" if torch.cuda.is_available() else None),
)
model.eval()

# ضمان وجود pad_token
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

print("Loaded:", REPO_ID, "| device:", model.device)


`torch_dtype` is deprecated! Use `dtype` instead!
2025-09-28 19:46:30.425122: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759088790.448661     140 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759088790.453754     140 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Loaded: HatemAhmed44/egiptura-adapter | device: cuda:0


In [7]:
import os
if "vectordb" not in globals():
    from pypdf import PdfReader
    from sentence_transformers import SentenceTransformer
    import faiss
    import numpy as np

    PDF_PATH = "/kaggle/input/egiptura-database/Egiptura Database.pdf"  # عدّل المسار لو لزم

    def load_pdf_text(path):
        reader = PdfReader(path)
        pages = [p.extract_text() or "" for p in reader.pages]
        text = "\n".join(pages)
        return text

    def chunk_text(text, chunk_size=900, overlap=150):
        chunks = []
        i = 0
        while i < len(text):
            chunk = text[i:i+chunk_size]
            chunks.append(chunk)
            i += (chunk_size - overlap)
        return [c.strip() for c in chunks if c.strip()]

    class SimpleVectorDB:
        def __init__(self, texts, embedder_name="sentence-transformers/all-MiniLM-L6-v2"):
            self.model = SentenceTransformer(embedder_name)
            self.texts = texts
            emb = self.model.encode(texts, convert_to_numpy=True, show_progress_bar=True, normalize_embeddings=True)
            self.index = faiss.IndexFlatIP(emb.shape[1])
            self.index.add(emb.astype(np.float32))

        def similarity_search(self, query, k=2):
            q = self.model.encode([query], convert_to_numpy=True, normalize_embeddings=True).astype(np.float32)
            D, I = self.index.search(q, k)
            class Doc:  # بسيط
                def __init__(self, page_content): self.page_content = page_content
            return [Doc(self.texts[i]) for i in I[0] if i >= 0]

    print("Building VectorDB from:", PDF_PATH)
    pdf_text = load_pdf_text(PDF_PATH)
    chunks = chunk_text(pdf_text)
    vectordb = SimpleVectorDB(chunks)
    print("VectorDB ready. Chunks:", len(chunks))
else:
    print("Using existing vectordb.")


Building VectorDB from: /kaggle/input/egiptura-database/Egiptura Database.pdf


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

VectorDB ready. Chunks: 137


In [22]:
# ===== Build / Load PDF index (FAISS + E5 multilingual) =====
!pip -q install langchain-community langchain-text-splitters faiss-cpu pypdf sentence-transformers

import os, glob, torch, re
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# --- Embeddings: E5 (متعدد اللغات) مع التطبيع + الـprefixات المناسبة
class E5Embeddings(HuggingFaceEmbeddings):
    def embed_documents(self, texts):
        return super().embed_documents([f"passage: {t}" for t in texts])
    def embed_query(self, text):
        return super().embed_query(f"query: {text}")

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
EMB_MODEL = "intfloat/multilingual-e5-base"  # جيّد للعربي/الإسباني/الإنجليزي
emb = E5Embeddings(
    model_name=EMB_MODEL,
    model_kwargs={"device": DEVICE},
    encode_kwargs={"normalize_embeddings": True},
)

INDEX_DIR = "egiptura_index"

def build_pdf_index(pdf_paths, persist_dir=INDEX_DIR, rebuild=False):
    # حمّل الصفحات مع metadata (المصدر + رقم الصفحة)
    docs = []
    for p in pdf_paths:
        try:
            loader = PyPDFLoader(p)
            pages  = loader.load()
            for i, d in enumerate(pages, start=1):
                d.metadata = d.metadata or {}
                d.metadata.update({"source": os.path.basename(p), "page": i})
            docs.extend(pages)
        except Exception as e:
            print(f"[!] تخطي {p}: {e}")

    if not docs:
        raise RuntimeError("لا يوجد مستندات PDF محمّلة. تأكد من المسارات.")

    # تقسيم ذكي
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=800, chunk_overlap=150,
        separators=["\n\n", "\n", " ", ""]
    )
    chunks = splitter.split_documents(docs)
    print(f"[i] Chunks: {len(chunks)}")

    # بناء فهرس FAISS
    vectordb = FAISS.from_documents(chunks, emb)
    if persist_dir:
        os.makedirs(persist_dir, exist_ok=True)
        vectordb.save_local(persist_dir)
        print(f"[✓] Saved index -> {persist_dir}")
    return vectordb

def load_or_build_index(pdf_glob_pattern="*.pdf", persist_dir=INDEX_DIR):
    # لو فيه index محفوظ: حمّله، وإلا ابنِ جديد
    if os.path.isdir(persist_dir) and os.path.exists(os.path.join(persist_dir, "index.faiss")):
        print(f"[i] Loading index from {persist_dir}")
        return FAISS.load_local(persist_dir, emb, allow_dangerous_deserialization=True)
    pdfs = sorted(glob.glob(pdf_glob_pattern))
    if not pdfs:
        raise RuntimeError("لم أجد PDF. عدّل الـglob مثلاً: 'data/**/*.pdf'")
    print(f"[i] Building index from {len(pdfs)} PDFs")
    return build_pdf_index(pdfs, persist_dir=persist_dir)

# شغّل واحدة من الاتنين (حسب مكان ملفاتك):
# vectordb = load_or_build_index("/kaggle/input/egiptura-database/**/*.pdf")  # مثال على كاجل
vectordb = load_or_build_index("**/*.pdf")  # عدّل الباترن لمسار ملفاتك

# --- (اختياري قوي) Reranker متعدد اللغات لتحسين النتائج
try:
    from sentence_transformers import CrossEncoder
    RERANKER = CrossEncoder("BAAI/bge-reranker-v2-m3", device=DEVICE)  # يدعم AR/ES/EN
    print("[✓] Reranker ready (BAAI/bge-reranker-v2-m3).")
except Exception as e:
    RERANKER = None
    print(f"[i] Reranker disabled: {e}")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


RuntimeError: لم أجد PDF. عدّل الـglob مثلاً: 'data/**/*.pdf'

In [20]:
# ===== Egiptura: رد سريع + لغة واحدة + سؤال متابعة واحد فقط (مع تنظيف وتصحيح) =====
import re, torch

# إعدادات عامة
_MAX_CTX   = 600
_MAX_NEW   = 160
_NO_REPEAT = 6
_REP_PEN   = 1.2

# كواشف/أنماط
_AR  = re.compile(r"[\u0600-\u06FF]")
_LAT = re.compile(r"[A-Za-zÁÉÍÓÚÜÑáéíóúüñ]")

# رموز وأسطر خاصة يجب تنظيفها
_SPECIAL         = re.compile(r"(?:<\|[^>|]+?\|>)|(?:</?s>)|(?:<<SYS>>|<</SYS>>)|(?:\[\s*/?\s*INST\s*\])|(?:<!\[CDATA\[|\]\]>)|(?:<\/?\w+\s*>)", re.IGNORECASE)
_PREFIX_LINE     = re.compile(r'(?im)^\s*(?:A|Ans|Resp|Rpta|Respuesta|Answer|الإجابة)\s*:?\s*')
_PREFIX_INLINE   = re.compile(r'(?i)\b(?:respuesta|answer|الإجابة)\s*:?\s*')
_DISCLAIMER      = re.compile(r'^\s*(?:nota|note|ملاحظة|تنبيه)\s*[:\-–]', re.IGNORECASE)
_EMPTY_LIST_LINE = re.compile(r'^\s*(?:[-–—•\u2022]|\(?\d{1,3}\)?[.)]?)\s*$')
_DECOR_LINE      = re.compile(r'^[\.\-=_~•·\u2022\s]{4,}$')
_EMOJI_RE        = re.compile(r"[\U0001F300-\U0001F6FF\U0001F900-\U0001FAFF\U0001F1E6-\U0001F1FF\u2600-\u26FF\u2700-\u27BF]")

# بقايا توكنات/رموز غريبة
_JUNK_TOK_RE     = re.compile(r"(?:</?s>|<s>|</s>|<<SYS>>|<</SYS>>|\[/?INST\])", re.I)
_JUNK_SYMBOLS_RE = re.compile(r"[<>\[\]\|/\\]{1,}")
_TAGGY_RE        = re.compile(r"</?[A-Za-z][^>]{0,20}>")

# كشف الأسعار ومنعها
_PRICE = re.compile(
    r"(?:\$\s?\d+(?:[,\.\s]\d+)*)|"
    r"(?:\d+\s?(?:USD|EUR|EGP|AED|SAR))|"
    r"(?:(?:USD|EUR|EGP|AED|SAR)\s?\d+(?:[,\.\s]\d+)*)|"
    r"(?:\d+\s?(?:دولار|يورو|جنيه|ريال|درهم))|"
    r"(?:precio|coste|costo|price)\s*(?:aprox\.?|estimado)?\s*:?[\s~]*\d+",
    re.IGNORECASE
)

# أسئلة/لغة
_QLINE_RE = re.compile(r"\s*(?:[¿].*|\S.*[?؟])\s*$")
_AR_CH    = r"[\u0600-\u06FF]"
_LAT_CH   = r"[A-Za-zÁÉÍÓÚÜÑáéíóúüñ]"

# ===================== Helpers =====================

def _detect_lang(q: str) -> str:
    s = q.strip().lower()
    if _AR.search(s): 
        return "ar"
    if re.search(r"[áéíóúüñ¿¡]", s) or any(w in s for w in (
        "hola","buenas","gracias","quiero","viaje","programa","itinerario",
        "semana","egipto","viajar","turismo","precio","coste","costo",
        "cuanto","cuánto"
    )):
        return "es"
    return "en"

def _wants_price(q: str) -> bool:
    s = q.lower()
    return any(k in s for k in ["price","cost","budget","how much","cuanto","cuánto","precio","coste","costo","tarifa","سعر","التكلفة","تكلفة","ميزانية"])

def _is_factoid(q: str) -> bool:
    ql = q.strip().lower()
    words = len(ql.split())
    return words <= 12 and bool(re.search(r"\b(quien|quién|que|qué|cuando|cuándo|donde|dónde|who|what|when|where|من|ما|متى|أين)\b", ql))

def _trim_tokens(text: str, max_tokens: int) -> str:
    global tok, tokenizer
    tok = tok if 'tok' in globals() else tokenizer
    ids = tok.encode(text, add_special_tokens=False)
    return text if len(ids) <= max_tokens else tok.decode(ids[:max_tokens], skip_special_tokens=True)

def _strip_decor(text: str) -> str:
    out = []
    for ln in text.splitlines():
        if _DECOR_LINE.match(ln.strip()):
            continue
        ln = re.sub(r'([.\-=_~•·\u2022])\1{2,}', r'\1\1', ln)
        out.append(ln)
    return "\n".join(out).strip()

def _strip_emojis(text: str) -> str:
    return _EMOJI_RE.sub("", text)

def _strip_token_garbage(t: str) -> str:
    t = _JUNK_TOK_RE.sub("", t)
    t = _TAGGY_RE.sub("", t)
    t = _JUNK_SYMBOLS_RE.sub(" ", t)
    t = re.sub(r"\s{2,}", " ", t)
    return t.strip()

def _clean(text: str) -> str:
    text = _SPECIAL.sub("", text)
    text = _PREFIX_LINE.sub("", text)
    text = _PREFIX_INLINE.sub("", text)
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\n{3,}", "\n\n", text).strip()
    text = "\n".join([ln for ln in text.splitlines() if not _DISCLAIMER.match(ln.strip())])
    text = "\n".join([ln for ln in text.splitlines() if not _EMPTY_LIST_LINE.match(ln)])
    # إزالة تكرارات
    lines, seen = [], set()
    for ln in text.splitlines():
        k = ln.strip()
        if not k:
            lines.append("")
            continue
        if k not in seen:
            seen.add(k); lines.append(ln)
    text = "\n".join(lines).strip()
    text = _strip_decor(text)
    return text

# فلترة لغة واحدة (أخفّ علشان الأرقام/الوحدات ما تتشالش)
def _enforce_lang(text: str, lang: str) -> str:
    def keep_line(line: str) -> bool:
        ar  = sum(1 for ch in line if _AR.match(ch))
        lat = sum(1 for ch in line if re.match(r"[A-Za-zÁÉÍÓÚÜÑáéíóúüñ]", ch))
        if ar == 0 and lat == 0:
            return True
        if lang == "ar": return ar  >= max(2, int(1.2 * lat))
        if lang == "es": return lat >= max(2, int(1.0 * ar))
        return lat >= max(2, int(1.0 * ar))
    return "\n".join([ln for ln in text.splitlines() if keep_line(ln)])

# إزالة ترجمات داخل أقواس بلغة مختلفة
_PARENS_RE = re.compile(r"\(([^)]{1,160})\)")
_BRACK_RE  = re.compile(r"\[([^\]]{1,160})\]")
def _strip_cross_parentheticals(text: str, lang: str) -> str:
    def bad_chunk(chunk: str) -> bool:
        has_ar  = bool(_AR.search(chunk))
        has_lat = bool(re.search(r"[A-Za-zÁÉÍÓÚÜÑáéíóúüñ]", chunk))
        if lang == "ar":  return has_lat and not has_ar
        if lang == "es":  return has_ar and not has_lat
        return has_ar
    def repl_parens(m): return "" if bad_chunk(m.group(1)) else m.group(0)
    def repl_brack(m):  return "" if bad_chunk(m.group(1)) else m.group(0)
    text = _PARENS_RE.sub(repl_parens, text)
    text = _BRACK_RE.sub(repl_brack, text)
    return text

def _strip_prices_if_needed(text: str, allow: bool) -> str:
    return text if allow else _PRICE.sub("▢", text).strip()

def _is_greeting(q: str) -> bool:
    return bool(re.search(r"\b(hola|buenas|hi|hello|hey|salut|مرحبا|اهلا|أهلاً|السلام عليكم)\b", q.strip().lower()))

def _normalize_question(q: str, lang: str) -> str:
    q = _strip_emojis(q)
    q = re.sub(r"[?؟]{2,}", "?", q)
    q = re.sub(r"[!¡]{2,}", "!", q)
    q = re.sub(r"[.\u2026]{3,}", "…", q)
    q = re.sub(r"\s{2,}", " ", q).strip()
    if lang == "ar":
        q = q.replace("¿", "").replace("?", "؟")
        if not q.endswith("؟"): q = q.rstrip("؟") + "؟"
    elif lang == "es":
        q = q.rstrip("?") + "?"
        if not q.startswith("¿"): q = "¿" + q
    else:
        q = q.replace("¿", "")
        q = q.rstrip("?") + "?"
    return q.strip()

def _line_matches_lang(line: str, lang: str) -> bool:
    has_ar  = bool(re.search(_AR_CH, line))
    has_lat = bool(re.search(_LAT_CH, line))
    if lang == "ar": return has_ar and not (has_lat and not has_ar)
    if lang == "es": return has_lat and not has_ar
    return has_lat and not has_ar

def _enforce_one_followup(text: str, lang: str) -> str:
    lines = [ln.strip() for ln in text.splitlines() if ln.strip()]
    keep_idx = None
    for i in range(len(lines) - 1, -1, -1):
        ln = lines[i]
        if _QLINE_RE.match(ln) and _line_matches_lang(ln, lang):
            keep_idx = i
            break
    cleaned = []
    for i, ln in enumerate(lines):
        if _QLINE_RE.match(ln):
            if i == keep_idx:
                cleaned.append(ln)
        else:
            cleaned.append(ln)
    if keep_idx is None:
        fallback = {
            "ar": "هل تود أن أضيف تفاصيل محددة؟",
            "es": "¿Quieres que añada algún detalle en concreto?",
            "en": "Would you like me to add any specific details?",
        }.get(lang, "Would you like me to add any specific details?")
        cleaned.append(fallback)
    else:
        if cleaned and _QLINE_RE.match(cleaned[-1]):
            cleaned[-1] = _normalize_question(cleaned[-1], lang)
    cleaned = [_strip_emojis(ln) for ln in cleaned]
    return "\n".join([ln for ln in cleaned if ln.strip()]).strip()

def _tone_open(text: str, lang: str, factoid: bool) -> str:
    t = text.strip()
    if not t:
        return {
            "ar": "مرحبًا! كيف أقدر أساعدك في رحلتك إلى مصر؟",
            "es": "¡Hola! ¿En qué puedo ayudarte con tu viaje a Egipto?",
            "en": "Hi! How can I help with your Egypt trip?",
        }.get(lang, "Hi! How can I help?")
    if factoid:
        return t
    openings = {"ar": "بكل سرور — ", "es": "¡Claro! ", "en": "Sure — "}
    if not t.lower().startswith(tuple(s.lower() for s in openings.values())):
        t = openings.get(lang, "Sure — ") + t
    return re.sub(r"\n{3,}", "\n\n", t)

# تصحيح حقائق شائعة
def _post_fix_facts(text: str, lang: str) -> str:
    if lang == "ar":
        text = re.sub(r"\bالمملكة\s+المصرية\b", "جمهورية مصر العربية", text)
    elif lang == "es":
        text = re.sub(r"\bReino\s+de\s+Egipto\b", "República Árabe de Egipto", text, flags=re.I)
    else:
        text = re.sub(r"\bKingdom of Egypt\b", "Arab Republic of Egypt", text, flags=re.I)
    return text

# إجابة جاهزة لبعض الأسئلة لتقليل الهلوسة (مثال: مساحة مصر)
def _fact_override(question: str, lang: str):
    q = question.strip().lower()
    if (lang == "ar" and ("كم مساحة مصر" in q or "مساحة مصر" in q)) \
       or (lang == "es" and re.search(r"(superficie|área).*(egipto)", q)) \
       or (lang == "en" and re.search(r"(area|size).*(egypt)", q)):
        if lang == "ar":
            return "بكل سرور — مساحة **جمهورية مصر العربية** حوالي **1,001,450 كم²** (≈ **1.0 مليون كم²**). هل تود أن أضيف تفاصيل محددة؟"
        elif lang == "es":
            return "¡Claro! La superficie de la **República Árabe de Egipto** es de **~1.001.450 km²** (≈ **1,0 millón km²**). ¿Quieres algún detalle más?"
        else:
            return "Sure — The area of the **Arab Republic of Egypt** is **about 1,001,450 km²** (≈ **1.0 million km²**). Would you like any specific details?"
    return None

# بناء البرومبت
def _build_prompts(lang: str, context: str, question: str):
    if lang == "ar":
        system = (
            "أنت مساعد Egiptura الودود والمحترف. أجب بالعربية الفصحى فقط وبلا أي مزج لغات أو ترجمات. "
            "أنت متخصص في السياحة داخل مصر وتاريخها والرحلات إلى مصر؛ لا تتحدث عن مواضيع لا صلة لها بمصر. "
            "نظّم الإجابة بنقاط/أيام عند الحاجة وتجنّب تكرار السؤال. "
            "إن لم تكن متأكدًا لا تُخمّن؛ اطلب أهم تفصيلة في جملة واحدة. "
            "لا تذكر الأسعار إلا إذا سُئلت صراحة. "
            "اكتفِ في النهاية بسؤال متابعة واحد قصير وبالعربية فقط عند الحاجة."
        )
        user = (f"السياق (اختياري):\n{context}\n\nسؤال:\n{question}\n\n"
                "أعطني الإجابة مباشرة بدون إعادة صياغة السؤال.")
    elif lang == "es":
        system = (
            "Eres el asistente de Egiptura. Responde SOLO en español, sin mezclar idiomas ni hacer traducciones. "
            "Te especializas en turismo, historia egipcia y viajes a Egipto. No hables de otros temas. "
            "Sé claro y conciso; usa viñetas/días cuando ayude. "
            "Si no estás seguro, no inventes: pide el dato clave en una sola frase. "
            "No menciones precios salvo que el usuario lo pida. "
            "Termina, si hace falta, con UNA sola pregunta breve en español."
        )
        user = (f"Contexto (opcional):\n{context}\n\nPregunta:\n{question}\n\n"
                "Dame la respuesta directa sin repetir el enunciado.")
    else:
        system = (
            "You are Egiptura’s assistant. Reply ONLY in English—no code-switching or translations. "
            "You specialize in tourism, Egyptian history, and trips to Egypt; don't discuss unrelated topics. "
            "Be concise; use bullets/day-by-day when helpful. "
            "If unsure, don’t fabricate; ask for the key detail in one short sentence. "
            "Do not mention pricing unless asked. "
            "If needed, end with ONE short follow-up question in English."
        )
        user = (f"Context (optional):\n{context}\n\nQuestion:\n{question}\n\n"
                "Return the direct answer without repeating the prompt.")
    return system, user

# ===================== الدالة الرئيسية =====================

def answer(question: str) -> str:
    """
    يُعيد ردًا سريعًا بلغة واحدة، نظيفًا، بدون أسعار (إلا لو سُئلت)،
    وينتهي بسؤال متابعة واحد فقط بنفس اللغة.
    """
    global tok, tokenizer, model, vectordb
    tok = tok if 'tok' in globals() else tokenizer

    user_q = question.strip()
    lang   = _detect_lang(user_q)

    # تحيات: رد فوري
    if _is_greeting(user_q):
        return {
            "ar": "مرحبًا! كيف أقدر أساعدك في رحلتك إلى مصر؟",
            "es": "¡Hola! ¿En qué puedo ayudarte con tu viaje a Egipto?",
            "en": "Hi! How can I help with your Egypt trip?",
        }.get(lang, "Hi! How can I help?")

    # إجابة جاهزة لبعض الحقائق (تقليل الهلوسة)
    ov = _fact_override(user_q, lang)
    if ov:
        return ov

    fact  = _is_factoid(user_q)
    wants = _wants_price(user_q)

    # RAG خفيف (اختياري)
    context = ""
    if 'vectordb' in globals() and vectordb:
        try:
            docs = vectordb.similarity_search(user_q, k=1) or []
            if docs:
                context = _trim_tokens("\n\n".join(d.page_content for d in docs), _MAX_CTX)
        except Exception:
            context = ""

    system, user = _build_prompts(lang, context, user_q)

    # ترميز
    use_template = hasattr(tok, "apply_chat_template") and getattr(tok, "chat_template", None)
    if use_template:
        messages = [{"role":"system","content":system},{"role":"user","content":user}]
        enc = tok.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to(model.device)
        if isinstance(enc, torch.Tensor):
            ins = {"input_ids": enc, "attention_mask": torch.ones_like(enc)}
        elif isinstance(enc, dict):
            ids = enc["input_ids"]; attn = enc.get("attention_mask", torch.ones_like(ids))
            ins = {"input_ids": ids, "attention_mask": attn}
        else:
            ids = enc.input_ids; attn = getattr(enc, "attention_mask", None) or torch.ones_like(ids)
            ins = {"input_ids": ids, "attention_mask": attn}
    else:
        prompt = f"{system}\n\nUSER:\n{user}\n\nASSISTANT:\n"
        enc = tok(prompt, return_tensors="pt", truncation=True).to(model.device)
        ins = {"input_ids": enc["input_ids"],
               "attention_mask": enc.get("attention_mask", torch.ones_like(enc["input_ids"]))}

    # IDs وممنوعات
    def _tid(s):
        try:
            t = tok.convert_tokens_to_ids(s)
            return t if isinstance(t, int) and t >= 0 else None
        except Exception:
            return None
    eos = tok.eos_token_id or _tid("</s>") or _tid("<|eot_id|>") or _tid("<|end_of_text|>")
    pad = tok.pad_token_id if tok.pad_token_id is not None else eos
    bad = [t for t in [_tid("</s>"), _tid("<s>"), _tid("[INST]"), _tid("[/INST]"),
                       _tid("<<SYS>>"), _tid("<</SYS>>"),
                       _tid("<|user|>"), _tid("<|assistant|>"), _tid("<|system|>")] if isinstance(t, int)]

    gen_kwargs = dict(
        max_new_tokens=min(_MAX_NEW, 96 if fact else _MAX_NEW),
        use_cache=True,
        eos_token_id=eos,
        pad_token_id=pad,
        no_repeat_ngram_size=_NO_REPEAT,
        repetition_penalty=_REP_PEN,
        num_beams=1,
        do_sample=False,  # سريع وثابت
    )
    if bad: gen_kwargs["bad_words_ids"] = [[i] for i in bad]

    with torch.inference_mode():
        out = model.generate(**ins, **gen_kwargs)

    gen_only = out[0, ins["input_ids"].shape[-1]:]
    text = tok.decode(gen_only, skip_special_tokens=True, clean_up_tokenization_spaces=True)

    # Post: نظافة كاملة + إزالة رموز + لغة واحدة + تصحيح حقائق + سؤال واحد + منع أسعار + نبرة لطيفة
    text = _clean(text)
    text = _strip_token_garbage(text)
    text = _strip_cross_parentheticals(text, lang)
    text = _enforce_lang(text, lang)
    text = _post_fix_facts(text, lang)
    text = _enforce_one_followup(text, lang)
    text = _strip_prices_if_needed(text, allow=wants)
    text = _tone_open(text, lang, factoid=fact)
    return text.strip()


INFO:     154.182.78.241:0 - "OPTIONS /generate HTTP/1.1" 200 OK
INFO:     154.182.78.241:0 - "POST /generate HTTP/1.1" 200 OK


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:     154.182.78.241:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     154.182.78.241:0 - "POST /generate HTTP/1.1" 200 OK


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:     154.182.78.241:0 - "POST /generate HTTP/1.1" 200 OK


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:     154.182.78.241:0 - "POST /generate HTTP/1.1" 200 OK


In [9]:
print(answer("¿Quién es el rey Ahmose?"))          # رد معرفي سريع، بدون قوائم فاضية ولا ملاحظات
print(answer("عايز برنامج 7 أيام في مصر"))          # نقاط مختصرة وبأسلوب لطيف
print(answer("How much for 1-week Egypt for 4?"))  # يسمح بالأسعار لأن السؤال طلبها صراحة


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


El faraón egipcio que lideró la batalla contra los hititas para liberar Egipto. ¿Cómo ha influido esta victoria en la historia? . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
¿Quieres que añada algún detalle en concreto?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

بكل سرور — برنامج 7 أيام: ديا على الأهرامات، يومين للبحث عن التماثيل، يومين للزيارة إلى أبو سمبل، يومين للرحلة البحرية من أسوان إلى القاهرة. انتهاءً بزيارة إلى مسجد سيدي محمد البارقي. يبدأ البرنامج من القاهرة ويستمر حتى القاهرة. يتم تقاسم الطعام مع العائلة المحلية. يمكنك اختيار بين أنواع الغرف المتوفرة. جميع الخدمات تشمل غرفة نوم مريحة ومطبخ محلي. تمتلك المدينة قصرا كبيرا يضم العديد من المساحات المفتوحة والغابات الطبيعية. يتم تنظيم
هل تود أن أضيف تفاصيل محددة؟


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Sure — For a single room at Categoría Gold during low season: $3,440. For a double room at Categoría Diamond during high season: $12,255. Next?


In [10]:
# ===== FastAPI + ngrok server for Egiptura =====
import os, re, time, socket, threading
import requests, nest_asyncio, uvicorn
from fastapi import FastAPI, Request, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pyngrok import ngrok, conf

# --- Config ---
API_KEY           = os.getenv("API_KEY", "secret123")
PREFERRED_PORT    = int(os.getenv("PORT", "8000"))
NGROK_AUTH_TOKEN  = (os.getenv("NGROK_AUTH_TOKEN") or os.getenv("NGROK_TOKEN") or "32JYankevjVACV9Gy2F0KQ2YhJ8_6Fne9bccQciyPpEtcJAKR").strip()

# --- App ---
app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],   # TODO: في الإنتاج حدِّد الدومينات المسموح بها
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.get("/alive")
async def alive():
    return {"ok": True}

@app.post("/generate")
async def generate(req: Request):
    # Auth
    auth = req.headers.get("authorization") or req.headers.get("Authorization") or ""
    if auth != f"Bearer {API_KEY}":
        raise HTTPException(status_code=401, detail="Unauthorized")

    # Read input
    try:
        data = await req.json()
    except Exception:
        raise HTTPException(status_code=400, detail="Invalid JSON body")

    q = (data.get("query") or data.get("prompt") or "").strip()
    if not q:
        raise HTTPException(status_code=400, detail="Missing 'query' (or 'prompt').")

    # Call model function (prefer 'answer', fallback to 'chat')
    try:
        if "answer" in globals():
            text = answer(q)
        elif "chat" in globals():
            text = chat(q)
        else:
            raise RuntimeError("No model function found: define answer(question) or chat(user_msg).")
    except ValueError as e:
        raise HTTPException(status_code=400, detail=str(e))
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"{type(e).__name__}: {e}")

    text = (text or "").strip()
    return {"response": text}

# --- Uvicorn in background thread ---
nest_asyncio.apply()
_server = None

def _port_is_free(port: int) -> bool:
    try:
        import socket as _s
        with _s.socket(_s.AF_INET, _s.SOCK_STREAM) as s:
            s.bind(("0.0.0.0", port))
            return True
    except OSError:
        return False

def pick_free_port(preferred: int = 8000) -> int:
    if _port_is_free(preferred):
        return preferred
    s = socket.socket()
    s.bind(("", 0))
    port = s.getsockname()[1]
    s.close()
    return port

def start_uvicorn_in_thread(port: int):
    global _server
    config = uvicorn.Config(app, host="0.0.0.0", port=port, log_level="info")
    _server = uvicorn.Server(config)
    th = threading.Thread(target=_server.run, daemon=True)
    th.start()
    # wait until alive
    for _ in range(60):
        try:
            r = requests.get(f"http://127.0.0.1:{port}/alive", timeout=0.5)
            if r.ok:
                break
        except Exception:
            pass
        time.sleep(0.25)
    return th

def stop_all_tunnels():
    try:
        for t in ngrok.get_tunnels():
            ngrok.disconnect(t.public_url)
    except Exception:
        pass

def start_ngrok_for_port(port: int) -> str:
    if not NGROK_AUTH_TOKEN:
        raise RuntimeError("NGROK_AUTH_TOKEN is not set. Put it in env then rerun.")
    conf.get_default().auth_token = NGROK_AUTH_TOKEN
    stop_all_tunnels()
    url = ngrok.connect(addr=f"http://127.0.0.1:{port}", proto="http").public_url
    print(f"🌍 Public URL: {url}")
    print(f"🔗 Swagger:   {url}/docs")
    return url

def run_server_with_ngrok(preferred_port: int = PREFERRED_PORT):
    port = pick_free_port(preferred_port)
    start_uvicorn_in_thread(port)
    return start_ngrok_for_port(port)

public_url = run_server_with_ngrok(PREFERRED_PORT)
print("✅ Ready! Use POST /generate with {'query': '...'} and Authorization: Bearer API_KEY")


INFO:     Started server process [140]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:55012 - "GET /alive HTTP/1.1" 200 OK
🌍 Public URL: https://cbfd92cc65a2.ngrok-free.app                                                  
🔗 Swagger:   https://cbfd92cc65a2.ngrok-free.app/docs
✅ Ready! Use POST /generate with {'query': '...'} and Authorization: Bearer API_KEY
INFO:     154.182.78.241:0 - "OPTIONS /generate HTTP/1.1" 200 OK
INFO:     154.182.78.241:0 - "POST /generate HTTP/1.1" 200 OK


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:     154.182.78.241:0 - "POST /generate HTTP/1.1" 200 OK


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:     154.182.78.241:0 - "POST /generate HTTP/1.1" 200 OK


In [ ]:
q="how much the journey in egypt for week cost in octoper with gold category for 10 persons?"
x=ask_question(q)
print(x)

In [ ]:
q="¿Quién es el rey Ahmos?"
x=ask_question(q)
print(x)
